# 🌍 Air Quality Dashboard Project - Complete Beginner's Guide

## Project Overview
We'll build a **resume-worthy Air Quality Dashboard** using real-world data that shows:
- 🗺️ Interactive maps of global air pollution
- 📊 Air quality trends over time
- 🏙️ City-by-city comparisons
- 💡 Health recommendations based on air quality

## Learning Objectives
By the end of this project, you'll know:
1. **Data Collection**: How to get real data from APIs
2. **Data Analysis**: Clean and explore data with Pandas
3. **Visualization**: Create interactive charts and maps
4. **Web Development**: Build dashboards with Streamlit
5. **Deployment**: Publish your app online for free

## Project Structure
- **Phase 1**: Setup and Data Exploration
- **Phase 2**: Build Dashboard Components
- **Phase 3**: Create Interactive Streamlit App
- **Phase 4**: Polish and Deploy

## 📦 Phase 1: Setup and Installation

First, let's install all the packages we'll need for this project. Don't worry about understanding everything yet - I'll explain each one as we use it!

In [2]:
# Install all required packages
# Run this cell first to install all dependencies

!pip install streamlit pandas requests plotly folium streamlit-folium
!pip install pydeck numpy matplotlib seaborn
!pip install python-dotenv

print("✅ All packages installed successfully!")
print("📝 Next: We'll learn what each package does as we use them")

📝 Next: We'll learn what each package does as we use them



## 📚 What do these packages do?

Let me explain each package we just installed:

**🌐 Data Collection & Processing:**
- **`requests`** - Get data from websites and APIs (like air quality data)
- **`pandas`** - Excel but for programmers! Clean and analyze data
- **`numpy`** - Fast math operations on data

**📊 Visualization & Charts:**
- **`plotly`** - Create beautiful, interactive charts and graphs
- **`matplotlib` & `seaborn`** - Traditional plotting libraries
- **`folium`** - Create interactive maps

**🖥️ Web Dashboard:**
- **`streamlit`** - Turn your Python code into a web app (our dashboard!)
- **`streamlit-folium`** - Embed maps in Streamlit
- **`pydeck`** - Advanced 3D visualizations

**🔧 Utilities:**
- **`python-dotenv`** - Manage secret keys safely

## 🔧 Step 1: Import Libraries and Test Setup

Now let's import all the libraries we'll use and test that everything is working correctly.

In [3]:
# Import all necessary libraries
import pandas as pd
import numpy as np
import requests
import streamlit as st
import plotly.express as px
import plotly.graph_objects as go
import folium
from folium import plugins
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import json
import time

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ All libraries imported successfully!")
print("🎯 Ready to start building our Air Quality Dashboard!")

# Test basic functionality
test_data = pd.DataFrame({
    'city': ['London', 'Paris', 'Tokyo'],
    'pollution': [25, 30, 15]
})

print("\n📊 Test data created:")
print(test_data)

✅ All libraries imported successfully!
🎯 Ready to start building our Air Quality Dashboard!

📊 Test data created:
     city  pollution
0  London         25
1   Paris         30
2   Tokyo         15


## 🌬️ Step 2: Understanding Air Quality Data

Before we build our dashboard, let's understand what air quality data looks like and what we'll be working with:

### **Key Air Quality Metrics:**
- **PM2.5** - Fine particles (< 2.5 micrometers) - most dangerous to health
- **PM10** - Larger particles (< 10 micrometers) 
- **NO2** - Nitrogen dioxide (from vehicles, industry)
- **SO2** - Sulfur dioxide (from fossil fuels)
- **O3** - Ozone (ground-level ozone is harmful)
- **CO** - Carbon monoxide

### **Air Quality Index (AQI) Scale:**
- **0-50**: Good (Green) 🟢
- **51-100**: Moderate (Yellow) 🟡  
- **101-150**: Unhealthy for Sensitive Groups (Orange) 🟠
- **151-200**: Unhealthy (Red) 🔴
- **201-300**: Very Unhealthy (Purple) 🟣
- **301+**: Hazardous (Maroon) ⚫

In [4]:
# Helper Functions for Air Quality Analysis

def calculate_aqi(pm25_value):
    """
    Calculate Air Quality Index (AQI) from PM2.5 values
    This is a simplified version - real AQI calculations are more complex
    """
    if pd.isna(pm25_value):
        return None
    
    if pm25_value <= 12:
        return int(((50 - 0) / (12 - 0)) * pm25_value + 0)
    elif pm25_value <= 35.4:
        return int(((100 - 51) / (35.4 - 12.1)) * (pm25_value - 12.1) + 51)
    elif pm25_value <= 55.4:
        return int(((150 - 101) / (55.4 - 35.5)) * (pm25_value - 35.5) + 101)
    elif pm25_value <= 150.4:
        return int(((200 - 151) / (150.4 - 55.5)) * (pm25_value - 55.5) + 151)
    elif pm25_value <= 250.4:
        return int(((300 - 201) / (250.4 - 150.5)) * (pm25_value - 150.5) + 201)
    else:
        return int(((400 - 301) / (350.4 - 250.5)) * (pm25_value - 250.5) + 301)

def get_aqi_category(aqi):
    """Get AQI category and color"""
    if pd.isna(aqi):
        return "No Data", "#CCCCCC"
    elif aqi <= 50:
        return "Good", "#00E400"
    elif aqi <= 100:
        return "Moderate", "#FFFF00"
    elif aqi <= 150:
        return "Unhealthy for Sensitive Groups", "#FF7E00"
    elif aqi <= 200:
        return "Unhealthy", "#FF0000"
    elif aqi <= 300:
        return "Very Unhealthy", "#8F3F97"
    else:
        return "Hazardous", "#7E0023"

def get_health_recommendation(aqi):
    """Get health recommendations based on AQI"""
    if pd.isna(aqi):
        return "No data available"
    elif aqi <= 50:
        return "🟢 Great day for outdoor activities!"
    elif aqi <= 100:
        return "🟡 Generally safe, but sensitive people should consider reducing prolonged outdoor exertion"
    elif aqi <= 150:
        return "🟠 Sensitive groups should reduce outdoor activities"
    elif aqi <= 200:
        return "🔴 Everyone should limit outdoor activities"
    elif aqi <= 300:
        return "🟣 Avoid outdoor activities - health alert!"
    else:
        return "⚫ Emergency conditions - stay indoors!"

# Test our functions
test_pm25 = 25.5
test_aqi = calculate_aqi(test_pm25)
category, color = get_aqi_category(test_aqi)
recommendation = get_health_recommendation(test_aqi)

print(f"✅ Helper functions created!")
print(f"📊 Test: PM2.5 = {test_pm25} → AQI = {test_aqi}")
print(f"📈 Category: {category}")
print(f"💡 Recommendation: {recommendation}")

✅ Helper functions created!
📊 Test: PM2.5 = 25.5 → AQI = 79
📈 Category: Moderate
💡 Recommendation: 🟡 Generally safe, but sensitive people should consider reducing prolonged outdoor exertion


## 📡 Step 3: Data Collection from OpenAQ API

Now let's learn how to get real air quality data from the internet! We'll use the **OpenAQ API** - a free, open-source platform that collects air quality data from governments and research organizations worldwide.

### **What is an API?**
- **API** = Application Programming Interface
- Think of it like a restaurant menu - you ask for specific data, and the API serves it to you
- OpenAQ's API gives us real-time air quality measurements from around the world

### **What we'll fetch:**
- Latest air quality measurements
- Historical trends
- Data for specific cities/countries
- Different pollutant types (PM2.5, PM10, NO2, etc.)

In [5]:
# Functions to fetch data from WAQI (World Air Quality Index) API
# WAQI provides free access to real-time air quality data from 11,000+ stations worldwide

def fetch_city_air_quality(city_name, api_token="demo"):
    """
    Fetch air quality data for a specific city from WAQI API
    
    Parameters:
    - city_name: Name of the city (e.g., 'london', 'beijing', 'new york')
    - api_token: API token (use 'demo' for testing, get your own from aqicn.org/data-platform/token/)
    
    Returns:
    - Dictionary with air quality data or None if error
    """
    base_url = "https://api.waqi.info/feed"
    url = f"{base_url}/{city_name}/?token={api_token}"
    
    try:
        print(f"🔄 Fetching air quality data for {city_name}...")
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        
        data = response.json()
        
        if data.get('status') == 'ok':
            return data.get('data')
        else:
            print(f"❌ API Error for {city_name}: {data.get('data', 'Unknown error')}")
            return None
            
    except requests.exceptions.RequestException as e:
        print(f"❌ Network error for {city_name}: {e}")
        return None

def fetch_multiple_cities_data(cities_list, api_token="demo"):
    """
    Fetch air quality data for multiple cities
    
    Parameters:
    - cities_list: List of city names
    - api_token: API token
    
    Returns:
    - List of dictionaries containing air quality data
    """
    all_data = []
    
    for city in cities_list:
        # Add small delay to be respectful to the API
        time.sleep(0.5)
        
        city_data = fetch_city_air_quality(city, api_token)
        if city_data:
            # Add city name to the data for easier processing
            city_data['query_city'] = city
            all_data.append(city_data)
    
    return all_data

def fetch_geolocation_air_quality(lat, lon, api_token="demo"):
    """
    Fetch air quality data for specific coordinates
    
    Parameters:
    - lat: Latitude
    - lon: Longitude  
    - api_token: API token
    
    Returns:
    - Dictionary with air quality data or None if error
    """
    base_url = "https://api.waqi.info/feed"
    url = f"{base_url}/geo:{lat};{lon}/?token={api_token}"
    
    try:
        print(f"🔄 Fetching air quality data for coordinates ({lat}, {lon})...")
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        
        data = response.json()
        
        if data.get('status') == 'ok':
            return data.get('data')
        else:
            print(f"❌ API Error for coordinates: {data.get('data', 'Unknown error')}")
            return None
            
    except requests.exceptions.RequestException as e:
        print(f"❌ Network error for coordinates: {e}")
        return None

def waqi_data_to_dataframe(waqi_data_list):
    """
    Convert WAQI API data to pandas DataFrame
    
    Parameters:
    - waqi_data_list: List of dictionaries from WAQI API
    
    Returns:
    - pandas DataFrame with standardized air quality data
    """
    if not waqi_data_list:
        print("⚠️ No WAQI data to convert")
        return pd.DataFrame()
    
    data_list = []
    
    for city_data in waqi_data_list:
        # Extract basic information
        base_info = {
            'city': city_data.get('city', {}).get('name', 'Unknown'),
            'query_city': city_data.get('query_city', 'Unknown'),
            'aqi': city_data.get('aqi', None),
            'latitude': None,
            'longitude': None,
            'date': city_data.get('time', {}).get('s', None),
            'url': city_data.get('city', {}).get('url', None)
        }
        
        # Extract coordinates if available
        geo = city_data.get('city', {}).get('geo', [])
        if len(geo) >= 2:
            base_info['latitude'] = float(geo[0]) if geo[0] else None
            base_info['longitude'] = float(geo[1]) if geo[1] else None
        
        # Extract individual pollutant measurements
        iaqi = city_data.get('iaqi', {})
        
        # For each pollutant, create a separate row
        pollutants_found = False
        for pollutant, data in iaqi.items():
            if isinstance(data, dict) and 'v' in data:
                pollutants_found = True
                row = base_info.copy()
                row.update({
                    'parameter': pollutant,
                    'value': data['v'],
                    'unit': 'AQI' if pollutant in ['pm25', 'pm10', 'no2', 'so2', 'co', 'o3'] else 'unknown'
                })
                data_list.append(row)
        
        # If no individual pollutants, add a row with overall AQI
        if not pollutants_found and base_info['aqi'] is not None:
            row = base_info.copy()
            row.update({
                'parameter': 'overall_aqi',
                'value': base_info['aqi'],
                'unit': 'AQI'
            })
            data_list.append(row)
    
    if not data_list:
        print("⚠️ No valid data extracted from WAQI response")
        return pd.DataFrame()
    
    df = pd.DataFrame(data_list)
    
    # Clean and process the data
    if not df.empty:
        # Convert date to datetime
        df['date'] = pd.to_datetime(df['date'], errors='coerce')
        
        # Convert value to numeric
        df['value'] = pd.to_numeric(df['value'], errors='coerce')
        
        # Add health categories based on overall AQI
        df['aqi_category'] = df['aqi'].apply(lambda x: get_aqi_category(x)[0] if pd.notna(x) else 'No Data')
        df['health_recommendation'] = df['aqi'].apply(lambda x: get_health_recommendation(x) if pd.notna(x) else 'No data available')
        
        # Extract country from city name if possible (basic parsing)
        df['country'] = df['city'].apply(lambda x: x.split(',')[-1].strip() if ',' in str(x) else 'Unknown')
    
    print(f"📊 Created DataFrame with {len(df)} rows and {len(df.columns)} columns")
    return df

# Test the WAQI API functions
print("🧪 Testing WAQI API connection...")
print("📝 Note: Using 'demo' token - get your own free token at aqicn.org/data-platform/token/")

# Test with a few major cities
test_cities = ['london', 'beijing', 'tokyo', 'new york', 'paris']
sample_data = fetch_multiple_cities_data(test_cities[:3])  # Test with first 3 cities

if sample_data:
    sample_df = waqi_data_to_dataframe(sample_data)
    print("\n📈 Sample data preview:")
    print(sample_df.head())
    print(f"\n🏙️ Cities in sample: {sample_df['city'].unique()}")
    print(f"📊 Parameters available: {sample_df['parameter'].unique()}")
else:
    print("⚠️ No data received - please check your internet connection")

🧪 Testing WAQI API connection...
📝 Note: Using 'demo' token - get your own free token at aqicn.org/data-platform/token/
🔄 Fetching air quality data for london...
🔄 Fetching air quality data for beijing...
🔄 Fetching air quality data for tokyo...
📊 Created DataFrame with 30 rows and 13 columns

📈 Sample data preview:
            city query_city  aqi   latitude   longitude                date  \
0  Shanghai (上海)     london   50  31.204737  121.448902 2025-07-19 20:00:00   
1  Shanghai (上海)     london   50  31.204737  121.448902 2025-07-19 20:00:00   
2  Shanghai (上海)     london   50  31.204737  121.448902 2025-07-19 20:00:00   
3  Shanghai (上海)     london   50  31.204737  121.448902 2025-07-19 20:00:00   
4  Shanghai (上海)     london   50  31.204737  121.448902 2025-07-19 20:00:00   

                               url parameter   value     unit aqi_category  \
0  https://aqicn.org/city/shanghai        co     4.6      AQI         Good   
1  https://aqicn.org/city/shanghai         h    74.

## 📊 Step 4: Create Sample Data for Development

Since API calls might be slow or fail sometimes, let's create sample data to test our dashboard. This is a common practice in development - always have backup data!

In [6]:
# Create comprehensive dataset using WAQI API data

def fetch_comprehensive_data(api_token="demo"):
    """
    Fetch comprehensive air quality data from major cities worldwide using WAQI API
    This function will be the backbone of our dashboard
    """
    
    # Define major cities from different continents for global coverage
    target_cities = [
        # North America
        'new york', 'los angeles', 'chicago', 'toronto', 'mexico city',
        
        # Europe  
        'london', 'paris', 'berlin', 'madrid', 'rome', 'amsterdam', 'stockholm',
        
        # Asia-Pacific
        'beijing', 'shanghai', 'tokyo', 'seoul', 'mumbai', 'delhi', 'bangkok', 
        'singapore', 'sydney', 'melbourne',
        
        # Middle East & Africa
        'dubai', 'cairo', 'cape town', 'johannesburg',
        
        # South America
        'sao paulo', 'rio de janeiro', 'buenos aires', 'santiago',
        
        # Additional major cities
        'moscow', 'istanbul', 'lagos', 'jakarta', 'manila'
    ]
    
    print("🌍 Starting comprehensive air quality data collection from WAQI...")
    print(f"🎯 Collecting data from {len(target_cities)} major cities worldwide")
    print("⏳ This will take a few minutes to gather global data...")
    
    # Fetch data for all cities
    all_data = fetch_multiple_cities_data(target_cities, api_token)
    
    if all_data:
        # Convert to DataFrame
        combined_df = waqi_data_to_dataframe(all_data)
        
        if not combined_df.empty:
            # Sort by AQI (worst first) for better analysis
            combined_df = combined_df.sort_values('aqi', ascending=False, na_position='last')
            
            print(f"\n🎉 Data collection complete!")
            print(f"📈 Total measurements: {len(combined_df)}")
            print(f"🏙️ Cities covered: {combined_df['city'].nunique()}")
            print(f"📊 Parameters: {combined_df['parameter'].unique()}")
            print(f"📅 Data timestamp: {combined_df['date'].max()}")
            
            return combined_df
        else:
            print("❌ No valid data was processed from API responses")
            return pd.DataFrame()
    else:
        print("❌ No data collected. Please check your internet connection.")
        return pd.DataFrame()

def analyze_global_dataset(df):
    """Analyze the collected global dataset with detailed insights"""
    if df.empty:
        print("⚠️ No data to analyze")
        return
    
    print("\n" + "="*60)
    print("🌍 GLOBAL AIR QUALITY ANALYSIS")
    print("="*60)
    
    # Basic statistics
    print(f"📈 Total Records: {len(df):,}")
    print(f"🏙️ Unique Cities: {df['city'].nunique()}")
    print(f"📊 Parameters Measured: {list(df['parameter'].unique())}")
    print(f"📅 Data Collection Time: {df['date'].max()}")
    
    # Overall AQI analysis
    overall_aqi_data = df[df['parameter'] == 'overall_aqi'].copy()
    if not overall_aqi_data.empty:
        print(f"\n📊 AQI Statistics:")
        print(f"   🌍 Global Average AQI: {overall_aqi_data['aqi'].mean():.1f}")
        print(f"   📉 Best AQI: {overall_aqi_data['aqi'].min():.0f}")
        print(f"   📈 Worst AQI: {overall_aqi_data['aqi'].max():.0f}")
    
    # Top polluted cities
    if not overall_aqi_data.empty:
        top_polluted = overall_aqi_data.nlargest(10, 'aqi')
        print(f"\n🔴 TOP 10 MOST POLLUTED CITIES:")
        for idx, row in top_polluted.iterrows():
            category, color = get_aqi_category(row['aqi'])
            print(f"   {row['city']}: AQI {row['aqi']:.0f} ({category})")
    
    # Cleanest cities
    if not overall_aqi_data.empty:
        least_polluted = overall_aqi_data.nsmallest(10, 'aqi')
        print(f"\n🟢 TOP 10 CLEANEST CITIES:")
        for idx, row in least_polluted.iterrows():
            category, color = get_aqi_category(row['aqi'])
            print(f"   {row['city']}: AQI {row['aqi']:.0f} ({category})")
    
    # Health category distribution
    if not overall_aqi_data.empty:
        category_counts = overall_aqi_data['aqi_category'].value_counts()
        print(f"\n📊 AIR QUALITY DISTRIBUTION:")
        for category, count in category_counts.items():
            percentage = (count / len(overall_aqi_data)) * 100
            print(f"   {category}: {count} cities ({percentage:.1f}%)")
    
    # Individual pollutant analysis
    pollutant_data = df[df['parameter'].isin(['pm25', 'pm10', 'no2', 'so2', 'co', 'o3'])].copy()
    if not pollutant_data.empty:
        print(f"\n🔬 POLLUTANT ANALYSIS:")
        for param in ['pm25', 'pm10', 'no2', 'so2', 'co', 'o3']:
            param_data = pollutant_data[pollutant_data['parameter'] == param]
            if not param_data.empty:
                print(f"   {param.upper()}: {len(param_data)} measurements, avg: {param_data['value'].mean():.1f}")

def get_api_token_info():
    """Provide information about getting a free API token"""
    print("\n" + "="*60)
    print("🔑 HOW TO GET YOUR FREE WAQI API TOKEN")
    print("="*60)
    print("1. Visit: https://aqicn.org/data-platform/token/")
    print("2. Enter your email address")
    print("3. You'll receive a free token instantly")
    print("4. Replace 'demo' with your token in the functions above")
    print("\n💡 Benefits of your own token:")
    print("   • Higher rate limits (1000 requests/second!)")
    print("   • More reliable access")
    print("   • Support the WAQI project")
    print("   • Completely FREE for non-commercial use")

# Execute data collection with the updated API
print("🚀 Starting real-time data collection from WAQI (World Air Quality Index)...")
print("🌐 WAQI provides data from 11,000+ monitoring stations worldwide!")

# Show token information
get_api_token_info()

# Collect the data
air_quality_data = fetch_comprehensive_data()

if not air_quality_data.empty:
    # Analyze the dataset
    analyze_global_dataset(air_quality_data)
    
    # Save to file for later use
    air_quality_data.to_csv('global_air_quality_data.csv', index=False)
    print(f"\n💾 Data saved to 'global_air_quality_data.csv'")
    print(f"🎯 Ready to build the dashboard with {len(air_quality_data)} real measurements!")
    print(f"🌍 Global air quality data collected from {air_quality_data['city'].nunique()} cities!")
    
else:
    print("❌ Failed to collect data. Please check your internet connection and try again.")
    print("💡 Tip: The 'demo' token has rate limits. Try again in a few minutes.")

🚀 Starting real-time data collection from WAQI (World Air Quality Index)...
🌐 WAQI provides data from 11,000+ monitoring stations worldwide!

🔑 HOW TO GET YOUR FREE WAQI API TOKEN
1. Visit: https://aqicn.org/data-platform/token/
2. Enter your email address
3. You'll receive a free token instantly
4. Replace 'demo' with your token in the functions above

💡 Benefits of your own token:
   • Higher rate limits (1000 requests/second!)
   • More reliable access
   • Support the WAQI project
   • Completely FREE for non-commercial use
🌍 Starting comprehensive air quality data collection from WAQI...
🎯 Collecting data from 35 major cities worldwide
⏳ This will take a few minutes to gather global data...
🔄 Fetching air quality data for new york...
🔄 Fetching air quality data for new york...
🔄 Fetching air quality data for los angeles...
🔄 Fetching air quality data for los angeles...
🔄 Fetching air quality data for chicago...
🔄 Fetching air quality data for chicago...
🔄 Fetching air quality data

## 📊 Step 5: Data Visualization Functions

Now let's create visualization functions that will work with our real API data. These will be the building blocks of our Streamlit dashboard.

In [7]:
# Updated Visualization Functions for WAQI Dashboard Data

def create_world_map_waqi(df):
    """Create interactive world map showing air quality by city using WAQI data"""
    if df.empty:
        return None
    
    # Filter for overall AQI data with valid coordinates
    aqi_data = df[
        (df['parameter'] == 'overall_aqi') & 
        (df['latitude'].notna()) & 
        (df['longitude'].notna()) &
        (df['aqi'].notna())
    ].copy()
    
    if aqi_data.empty:
        return None
    
    # Create the map
    world_map = folium.Map(location=[20, 0], zoom_start=2, tiles='OpenStreetMap')
    
    for idx, row in aqi_data.iterrows():
        # Get color based on AQI
        category, color = get_aqi_category(row['aqi'])
        
        folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            radius=10,
            popup=folium.Popup(f"""
            <div style="font-family: Arial; width: 200px;">
                <h4 style="margin: 0; color: #333;">{row['city']}</h4>
                <hr style="margin: 5px 0;">
                <p style="margin: 5px 0;"><strong>AQI:</strong> {row['aqi']:.0f}</p>
                <p style="margin: 5px 0;"><strong>Status:</strong> {category}</p>
                <p style="margin: 5px 0;"><strong>Health Advice:</strong></p>
                <p style="margin: 5px 0; font-size: 12px;">{row['health_recommendation']}</p>
                <p style="margin: 5px 0; font-size: 10px; color: #666;">
                    Updated: {row['date'].strftime('%Y-%m-%d %H:%M') if pd.notna(row['date']) else 'N/A'}
                </p>
            </div>
            """, max_width=250),
            color='black',
            weight=2,
            fillColor=color,
            fillOpacity=0.8
        ).add_to(world_map)
    
    # Add a legend
    legend_html = """
    <div style="position: fixed; 
                bottom: 50px; left: 50px; width: 200px; height: 140px; 
                background-color: white; border:2px solid grey; z-index:9999; 
                font-size:14px; padding: 10px">
    <h4 style="margin-top:0;">Air Quality Index</h4>
    <p><span style="color:#00E400;">●</span> Good (0-50)</p>
    <p><span style="color:#FFFF00;">●</span> Moderate (51-100)</p>
    <p><span style="color:#FF7E00;">●</span> Unhealthy for Sensitive (101-150)</p>
    <p><span style="color:#FF0000;">●</span> Unhealthy (151-200)</p>
    <p><span style="color:#8F3F97;">●</span> Very Unhealthy (201-300)</p>
    <p><span style="color:#7E0023;">●</span> Hazardous (300+)</p>
    </div>
    """
    world_map.get_root().html.add_child(folium.Element(legend_html))
    
    return world_map

def create_city_ranking_chart_waqi(df, top_n=20):
    """Create bar chart ranking cities by AQI levels"""
    if df.empty:
        return None
    
    aqi_data = df[df['parameter'] == 'overall_aqi'].copy()
    if aqi_data.empty:
        return None
    
    # Sort by AQI and get top N
    top_cities = aqi_data.nlargest(top_n, 'aqi')
    
    # Create color mapping based on AQI categories
    color_map = {
        'Good': '#00E400',
        'Moderate': '#FFFF00', 
        'Unhealthy for Sensitive Groups': '#FF7E00',
        'Unhealthy': '#FF0000',
        'Very Unhealthy': '#8F3F97',
        'Hazardous': '#7E0023',
        'No Data': '#CCCCCC'
    }
    
    top_cities['color'] = top_cities['aqi_category'].map(color_map)
    
    fig = px.bar(
        top_cities,
        x='aqi',
        y='city',
        orientation='h',
        title=f'Top {top_n} Cities by Air Quality Index (Worst First)',
        labels={'aqi': 'Air Quality Index (AQI)', 'city': 'City'},
        color='aqi_category',
        color_discrete_map=color_map,
        hover_data={'aqi': True, 'aqi_category': True, 'date': True}
    )
    
    fig.update_layout(
        height=600,
        yaxis={'categoryorder': 'total ascending'},
        showlegend=True,
        legend_title="AQI Category"
    )
    
    # Add AQI threshold lines
    fig.add_vline(x=50, line_dash="dash", line_color="green", annotation_text="Good")
    fig.add_vline(x=100, line_dash="dash", line_color="orange", annotation_text="Moderate")
    fig.add_vline(x=150, line_dash="dash", line_color="red", annotation_text="Unhealthy")
    
    return fig

def create_cleanest_cities_chart_waqi(df, top_n=15):
    """Create bar chart showing cleanest cities"""
    if df.empty:
        return None
    
    aqi_data = df[df['parameter'] == 'overall_aqi'].copy()
    if aqi_data.empty:
        return None
    
    # Sort by AQI and get cleanest cities
    cleanest_cities = aqi_data.nsmallest(top_n, 'aqi')
    
    color_map = {
        'Good': '#00E400',
        'Moderate': '#FFFF00', 
        'Unhealthy for Sensitive Groups': '#FF7E00',
        'Unhealthy': '#FF0000',
        'Very Unhealthy': '#8F3F97',
        'Hazardous': '#7E0023',
        'No Data': '#CCCCCC'
    }
    
    fig = px.bar(
        cleanest_cities,
        x='aqi',
        y='city',
        orientation='h',
        title=f'Top {top_n} Cities with Best Air Quality',
        labels={'aqi': 'Air Quality Index (AQI)', 'city': 'City'},
        color='aqi_category',
        color_discrete_map=color_map,
        hover_data={'aqi': True, 'aqi_category': True, 'date': True}
    )
    
    fig.update_layout(
        height=500,
        yaxis={'categoryorder': 'total descending'},
        showlegend=True,
        legend_title="AQI Category"
    )
    
    return fig

def create_pollutant_comparison_waqi(df, selected_cities=None):
    """Create chart comparing different pollutants across cities"""
    if df.empty:
        return None
    
    # Filter for specific pollutants
    pollutant_data = df[df['parameter'].isin(['pm25', 'pm10', 'no2', 'so2', 'co', 'o3'])].copy()
    
    if selected_cities:
        pollutant_data = pollutant_data[pollutant_data['city'].isin(selected_cities)]
    
    if pollutant_data.empty:
        return None
    
    # Create grouped bar chart
    fig = px.bar(
        pollutant_data,
        x='city',
        y='value',
        color='parameter',
        title='Pollutant Levels by City',
        labels={'value': 'AQI Value', 'city': 'City', 'parameter': 'Pollutant'},
        barmode='group'
    )
    
    fig.update_layout(
        height=500,
        xaxis_tickangle=-45
    )
    
    return fig

def create_aqi_distribution_waqi(df):
    """Create histogram showing global AQI distribution"""
    if df.empty:
        return None
    
    aqi_data = df[df['parameter'] == 'overall_aqi'].copy()
    if aqi_data.empty:
        return None
    
    fig = px.histogram(
        aqi_data,
        x='aqi',
        nbins=25,
        title='Global Air Quality Index Distribution',
        labels={'aqi': 'Air Quality Index', 'count': 'Number of Cities'},
        color_discrete_sequence=['#1f77b4']
    )
    
    # Add AQI category lines with labels
    fig.add_vline(x=50, line_dash="dash", line_color="green", 
                  annotation_text="Good/Moderate", annotation_position="top")
    fig.add_vline(x=100, line_dash="dash", line_color="orange", 
                  annotation_text="Moderate/Unhealthy", annotation_position="top")
    fig.add_vline(x=150, line_dash="dash", line_color="red", 
                  annotation_text="Unhealthy/Very Unhealthy", annotation_position="top")
    fig.add_vline(x=200, line_dash="dash", line_color="purple", 
                  annotation_text="Very Unhealthy/Hazardous", annotation_position="top")
    
    fig.update_layout(height=400)
    return fig

def create_global_stats_summary(df):
    """Create summary statistics for the dashboard"""
    if df.empty:
        return None
    
    aqi_data = df[df['parameter'] == 'overall_aqi'].copy()
    if aqi_data.empty:
        return {}
    
    stats = {
        'total_cities': len(aqi_data),
        'avg_aqi': aqi_data['aqi'].mean(),
        'best_city': aqi_data.loc[aqi_data['aqi'].idxmin(), 'city'] if not aqi_data.empty else 'N/A',
        'best_aqi': aqi_data['aqi'].min(),
        'worst_city': aqi_data.loc[aqi_data['aqi'].idxmax(), 'city'] if not aqi_data.empty else 'N/A',
        'worst_aqi': aqi_data['aqi'].max(),
        'good_cities': len(aqi_data[aqi_data['aqi'] <= 50]),
        'unhealthy_cities': len(aqi_data[aqi_data['aqi'] > 150])
    }
    
    return stats

print("✅ Updated visualization functions created for WAQI data!")
print("📊 New functions available:")
print("  - create_world_map_waqi(): Interactive global map with AQI data")
print("  - create_city_ranking_chart_waqi(): City pollution ranking")
print("  - create_cleanest_cities_chart_waqi(): Best air quality cities") 
print("  - create_pollutant_comparison_waqi(): Multi-pollutant comparison")
print("  - create_aqi_distribution_waqi(): Global AQI histogram")
print("  - create_global_stats_summary(): Dashboard summary statistics")
print("\n🎯 Ready to create stunning visualizations with real WAQI data!")

✅ Updated visualization functions created for WAQI data!
📊 New functions available:
  - create_world_map_waqi(): Interactive global map with AQI data
  - create_city_ranking_chart_waqi(): City pollution ranking
  - create_cleanest_cities_chart_waqi(): Best air quality cities
  - create_pollutant_comparison_waqi(): Multi-pollutant comparison
  - create_aqi_distribution_waqi(): Global AQI histogram
  - create_global_stats_summary(): Dashboard summary statistics

🎯 Ready to create stunning visualizations with real WAQI data!


## 🖥️ Step 6: Create Streamlit Dashboard

Now let's create the actual Streamlit dashboard! This will be a separate Python file that we'll run to launch our web application.

In [ ]:
# Create Streamlit Dashboard Application File

dashboard_code = '''
import streamlit as st
import pandas as pd
import numpy as np
import requests
import plotly.express as px
import plotly.graph_objects as go
import folium
from streamlit_folium import st_folium
import time
from datetime import datetime

# Configure Streamlit page
st.set_page_config(
    page_title="🌍 Global Air Quality Dashboard",
    page_icon="🌬️",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Custom CSS for better styling
st.markdown("""
<style>
    .main-header {
        font-size: 3rem;
        color: #1f77b4;
        text-align: center;
        margin-bottom: 2rem;
    }
    .metric-card {
        background-color: #f0f2f6;
        padding: 1rem;
        border-radius: 0.5rem;
        border-left: 4px solid #1f77b4;
    }
    .aqi-good { color: #00E400; font-weight: bold; }
    .aqi-moderate { color: #FFFF00; font-weight: bold; }
    .aqi-unhealthy { color: #FF0000; font-weight: bold; }
    .aqi-very-unhealthy { color: #8F3F97; font-weight: bold; }
    .aqi-hazardous { color: #7E0023; font-weight: bold; }
</style>
""", unsafe_allow_html=True)

# Helper functions (copied from notebook)
def calculate_aqi(pm25_value):
    """Calculate Air Quality Index (AQI) from PM2.5 values"""
    if pd.isna(pm25_value):
        return None
    
    if pm25_value <= 12:
        return int(((50 - 0) / (12 - 0)) * pm25_value + 0)
    elif pm25_value <= 35.4:
        return int(((100 - 51) / (35.4 - 12.1)) * (pm25_value - 12.1) + 51)
    elif pm25_value <= 55.4:
        return int(((150 - 101) / (55.4 - 35.5)) * (pm25_value - 35.5) + 101)
    elif pm25_value <= 150.4:
        return int(((200 - 151) / (150.4 - 55.5)) * (pm25_value - 55.5) + 151)
    elif pm25_value <= 250.4:
        return int(((300 - 201) / (250.4 - 150.5)) * (pm25_value - 150.5) + 201)
    else:
        return int(((400 - 301) / (350.4 - 250.5)) * (pm25_value - 250.5) + 301)

def get_aqi_category(aqi):
    """Get AQI category and color"""
    if pd.isna(aqi):
        return "No Data", "#CCCCCC"
    elif aqi <= 50:
        return "Good", "#00E400"
    elif aqi <= 100:
        return "Moderate", "#FFFF00"
    elif aqi <= 150:
        return "Unhealthy for Sensitive Groups", "#FF7E00"
    elif aqi <= 200:
        return "Unhealthy", "#FF0000"
    elif aqi <= 300:
        return "Very Unhealthy", "#8F3F97"
    else:
        return "Hazardous", "#7E0023"

def get_health_recommendation(aqi):
    """Get health recommendations based on AQI"""
    if pd.isna(aqi):
        return "No data available"
    elif aqi <= 50:
        return "🟢 Great day for outdoor activities!"
    elif aqi <= 100:
        return "🟡 Generally safe, but sensitive people should consider reducing prolonged outdoor exertion"
    elif aqi <= 150:
        return "🟠 Sensitive groups should reduce outdoor activities"
    elif aqi <= 200:
        return "🔴 Everyone should limit outdoor activities"
    elif aqi <= 300:
        return "🟣 Avoid outdoor activities - health alert!"
    else:
        return "⚫ Emergency conditions - stay indoors!"

@st.cache_data(ttl=1800)  # Cache for 30 minutes
def fetch_city_air_quality(city_name, api_token="demo"):
    """Fetch air quality data for a specific city from WAQI API"""
    base_url = "https://api.waqi.info/feed"
    url = f"{base_url}/{city_name}/?token={api_token}"
    
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        data = response.json()
        
        if data.get('status') == 'ok':
            return data.get('data')
        else:
            return None
    except:
        return None

@st.cache_data(ttl=1800)  # Cache for 30 minutes
def fetch_multiple_cities_data(cities_list, api_token="demo"):
    """Fetch air quality data for multiple cities"""
    all_data = []
    progress_bar = st.progress(0)
    status_text = st.empty()
    
    for i, city in enumerate(cities_list):
        status_text.text(f'Fetching data for {city}...')
        progress_bar.progress((i + 1) / len(cities_list))
        
        city_data = fetch_city_air_quality(city, api_token)
        if city_data:
            city_data['query_city'] = city
            all_data.append(city_data)
        
        time.sleep(0.3)  # Be respectful to the API
    
    status_text.text('Data collection complete!')
    time.sleep(1)
    status_text.empty()
    progress_bar.empty()
    
    return all_data

def waqi_data_to_dataframe(waqi_data_list):
    """Convert WAQI API data to pandas DataFrame"""
    if not waqi_data_list:
        return pd.DataFrame()
    
    data_list = []
    
    for city_data in waqi_data_list:
        base_info = {
            'city': city_data.get('city', {}).get('name', 'Unknown'),
            'query_city': city_data.get('query_city', 'Unknown'),
            'aqi': city_data.get('aqi', None),
            'latitude': None,
            'longitude': None,
            'date': city_data.get('time', {}).get('s', None),
            'url': city_data.get('city', {}).get('url', None)
        }
        
        geo = city_data.get('city', {}).get('geo', [])
        if len(geo) >= 2:
            base_info['latitude'] = float(geo[0]) if geo[0] else None
            base_info['longitude'] = float(geo[1]) if geo[1] else None
        
        iaqi = city_data.get('iaqi', {})
        
        pollutants_found = False
        for pollutant, data in iaqi.items():
            if isinstance(data, dict) and 'v' in data:
                pollutants_found = True
                row = base_info.copy()
                row.update({
                    'parameter': pollutant,
                    'value': data['v'],
                    'unit': 'AQI'
                })
                data_list.append(row)
        
        if not pollutants_found and base_info['aqi'] is not None:
            row = base_info.copy()
            row.update({
                'parameter': 'overall_aqi',
                'value': base_info['aqi'],
                'unit': 'AQI'
            })
            data_list.append(row)
    
    if not data_list:
        return pd.DataFrame()
    
    df = pd.DataFrame(data_list)
    
    if not df.empty:
        df['date'] = pd.to_datetime(df['date'], errors='coerce')
        df['value'] = pd.to_numeric(df['value'], errors='coerce')
        df['aqi_category'] = df['aqi'].apply(lambda x: get_aqi_category(x)[0] if pd.notna(x) else 'No Data')
        df['health_recommendation'] = df['aqi'].apply(lambda x: get_health_recommendation(x) if pd.notna(x) else 'No data available')
        df['country'] = df['city'].apply(lambda x: x.split(',')[-1].strip() if ',' in str(x) else 'Unknown')
    
    return df

def create_world_map(df):
    """Create interactive world map"""
    if df.empty:
        return None
    
    aqi_data = df[
        (df['parameter'] == 'overall_aqi') & 
        (df['latitude'].notna()) & 
        (df['longitude'].notna()) &
        (df['aqi'].notna())
    ].copy()
    
    if aqi_data.empty:
        return None
    
    m = folium.Map(location=[20, 0], zoom_start=2)
    
    for idx, row in aqi_data.iterrows():
        category, color = get_aqi_category(row['aqi'])
        
        folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            radius=8,
            popup=f"{row['city']}<br>AQI: {row['aqi']}<br>{category}",
            color='black',
            weight=1,
            fillColor=color,
            fillOpacity=0.7
        ).add_to(m)
    
    return m

def main():
    # Header
    st.markdown('<h1 class="main-header">🌍 Global Air Quality Dashboard</h1>', unsafe_allow_html=True)
    st.markdown("**Real-time air quality data from cities worldwide**")
    
    # Sidebar
    st.sidebar.header("🎛️ Dashboard Controls")
    
    # API Token input
    api_token = st.sidebar.text_input(
        "🔑 WAQI API Token", 
        value="demo", 
        help="Get your free token at aqicn.org/data-platform/token/"
    )
    
    # City selection
    default_cities = [
        'london', 'paris', 'berlin', 'new york', 'los angeles', 'beijing', 
        'tokyo', 'mumbai', 'delhi', 'sydney', 'moscow', 'cairo'
    ]
    
    selected_cities = st.sidebar.multiselect(
        "🏙️ Select Cities",
        options=[
            'london', 'paris', 'berlin', 'madrid', 'rome', 'amsterdam',
            'new york', 'los angeles', 'chicago', 'toronto', 'mexico city',
            'beijing', 'shanghai', 'tokyo', 'seoul', 'mumbai', 'delhi', 'bangkok',
            'sydney', 'melbourne', 'moscow', 'istanbul', 'dubai', 'cairo',
            'sao paulo', 'rio de janeiro', 'buenos aires', 'santiago'
        ],
        default=default_cities[:8]
    )
    
    if st.sidebar.button("🔄 Refresh Data"):
        st.cache_data.clear()
    
    # Load data
    if selected_cities:
        with st.spinner('Fetching real-time air quality data...'):
            raw_data = fetch_multiple_cities_data(selected_cities, api_token)
            df = waqi_data_to_dataframe(raw_data)
        
        if not df.empty:
            # Summary metrics
            aqi_data = df[df['parameter'] == 'overall_aqi']
            
            if not aqi_data.empty:
                col1, col2, col3, col4 = st.columns(4)
                
                with col1:
                    st.metric("🏙️ Cities Monitored", len(aqi_data))
                
                with col2:
                    avg_aqi = aqi_data['aqi'].mean()
                    st.metric("📊 Average AQI", f"{avg_aqi:.0f}")
                
                with col3:
                    best_city = aqi_data.loc[aqi_data['aqi'].idxmin()]
                    st.metric("🟢 Best Air Quality", f"{best_city['city']} ({best_city['aqi']:.0f})")
                
                with col4:
                    worst_city = aqi_data.loc[aqi_data['aqi'].idxmax()]
                    st.metric("🔴 Worst Air Quality", f"{worst_city['city']} ({worst_city['aqi']:.0f})")
            
            # Main content tabs
            tab1, tab2, tab3 = st.tabs(["🗺️ World Map", "📊 City Rankings", "📈 Analysis"])
            
            with tab1:
                st.subheader("🗺️ Global Air Quality Map")
                world_map = create_world_map(df)
                if world_map:
                    map_data = st_folium(world_map, width=1200, height=500)
            
            with tab2:
                st.subheader("📊 City Air Quality Rankings")
                
                if not aqi_data.empty:
                    # Create ranking chart
                    aqi_sorted = aqi_data.sort_values('aqi', ascending=False)
                    
                    fig = px.bar(
                        aqi_sorted,
                        x='aqi',
                        y='city',
                        orientation='h',
                        title='Cities Ranked by Air Quality Index (Worst First)',
                        color='aqi_category',
                        color_discrete_map={
                            'Good': '#00E400',
                            'Moderate': '#FFFF00',
                            'Unhealthy for Sensitive Groups': '#FF7E00',
                            'Unhealthy': '#FF0000',
                            'Very Unhealthy': '#8F3F97',
                            'Hazardous': '#7E0023'
                        }
                    )
                    fig.update_layout(height=600, yaxis={'categoryorder': 'total ascending'})
                    st.plotly_chart(fig, use_container_width=True)
                    
                    # City details table
                    st.subheader("📋 Detailed City Information")
                    display_df = aqi_data[['city', 'aqi', 'aqi_category', 'health_recommendation']].copy()
                    display_df.columns = ['City', 'AQI', 'Category', 'Health Recommendation']
                    st.dataframe(display_df, use_container_width=True)
            
            with tab3:
                st.subheader("📈 Air Quality Analysis")
                
                if not aqi_data.empty:
                    col1, col2 = st.columns(2)
                    
                    with col1:
                        # AQI distribution
                        fig_hist = px.histogram(
                            aqi_data, 
                            x='aqi', 
                            nbins=15,
                            title='Distribution of AQI Values'
                        )
                        st.plotly_chart(fig_hist, use_container_width=True)
                    
                    with col2:
                        # Category pie chart
                        category_counts = aqi_data['aqi_category'].value_counts()
                        fig_pie = px.pie(
                            values=category_counts.values,
                            names=category_counts.index,
                            title='Air Quality Categories Distribution'
                        )
                        st.plotly_chart(fig_pie, use_container_width=True)
                    
                    # Pollutant comparison
                    st.subheader("🔬 Pollutant Levels Comparison")
                    pollutant_data = df[df['parameter'].isin(['pm25', 'pm10', 'no2', 'so2', 'co', 'o3'])]
                    
                    if not pollutant_data.empty:
                        fig_pollutants = px.bar(
                            pollutant_data,
                            x='city',
                            y='value',
                            color='parameter',
                            title='Pollutant Levels by City',
                            barmode='group'
                        )
                        fig_pollutants.update_layout(xaxis_tickangle=-45)
                        st.plotly_chart(fig_pollutants, use_container_width=True)
        
        else:
            st.error("No data available. Please check your internet connection or try different cities.")
    
    else:
        st.info("👆 Please select cities from the sidebar to view air quality data.")
    
    # Footer
    st.markdown("---")
    st.markdown("**Data Source:** [World Air Quality Index](https://waqi.info/) | **Built with:** Streamlit & Python")
    st.markdown("**Note:** Air quality data is updated in real-time from monitoring stations worldwide.")

if __name__ == "__main__":
    main()
'''

# Save the dashboard code to a file
with open('air_quality_dashboard.py', 'w', encoding='utf-8') as f:
    f.write(dashboard_code)

print("✅ Streamlit dashboard code created!")
print("📁 File saved as: air_quality_dashboard.py")
print("\n🚀 To run the dashboard:")
print("1. Open terminal/command prompt")
print("2. Navigate to this directory")
print("3. Run: streamlit run air_quality_dashboard.py")
print("4. Your browser will open with the dashboard!")
print("\n💡 The dashboard includes:")
print("   🗺️ Interactive world map")
print("   📊 City rankings and comparisons") 
print("   📈 Data analysis and visualizations")
print("   🔄 Real-time data updates")
print("   📱 Responsive design")

✅ Streamlit dashboard code created!
📁 File saved as: air_quality_dashboard.py

🚀 To run the dashboard:
1. Open terminal/command prompt
2. Navigate to this directory
3. Run: streamlit run air_quality_dashboard.py
4. Your browser will open with the dashboard!

💡 The dashboard includes:
   🗺️ Interactive world map
   📊 City rankings and comparisons
   📈 Data analysis and visualizations
   🔄 Real-time data updates
   📱 Responsive design


✅ Streamlit dashboard code created!
📁 File saved as: air_quality_dashboard.py

🚀 To run the dashboard:
1. Open terminal/command prompt
2. Navigate to this directory
3. Run: streamlit run air_quality_dashboard.py
4. Your browser will open with the dashboard!

💡 The dashboard includes:
   🗺️ Interactive world map
   📊 City rankings and comparisons
   📈 Data analysis and visualizations
   🔄 Real-time data updates
   📱 Responsive design


: 

## 🚀 Step 7: Run and Deploy Your Dashboard

### **Running Locally**
1. **Save the dashboard code** (already done above)
2. **Install Streamlit** (if not already installed):
   ```bash
   pip install streamlit streamlit-folium
   ```
3. **Run the dashboard**:
   ```bash
   streamlit run air_quality_dashboard.py
   ```
4. **Open your browser** - Streamlit will automatically open your dashboard!

### **Free Deployment Options**

#### **Option 1: Streamlit Community Cloud (Recommended)**
1. **Push to GitHub**:
   - Create a new repository on GitHub
   - Upload your `air_quality_dashboard.py` file
   - Create a `requirements.txt` file with dependencies

2. **Deploy on Streamlit Cloud**:
   - Visit [share.streamlit.io](https://share.streamlit.io)
   - Connect your GitHub account
   - Select your repository
   - Deploy with one click!

#### **Option 2: Heroku (Alternative)**
- Free tier available
- Requires a bit more setup
- Good for learning deployment

### **Get Your Own API Token**
- Visit: [aqicn.org/data-platform/token](https://aqicn.org/data-platform/token/)
- Enter your email
- Get instant free access to 1000 requests/second!
- Replace "demo" in the code with your token

### **Resume Impact**
**Project Title:** "Global Air Quality Analytics Dashboard"

**Description:** "Built a real-time web application analyzing air quality from 11,000+ monitoring stations worldwide. Features interactive geospatial visualizations, city rankings, and health recommendations. Deployed using Streamlit Cloud with automated data pipelines."

**Technologies:** Python, Pandas, Streamlit, Plotly, Folium, REST APIs, Data Visualization, Web Development

## 🎉 Congratulations! Your Air Quality Dashboard is Complete!

### **What You've Built**
✅ **Data Collection Pipeline** - Real-time API integration with WAQI  
✅ **Data Processing** - Pandas-based data cleaning and analysis  
✅ **Interactive Visualizations** - Maps, charts, and analytics  
✅ **Web Dashboard** - Professional Streamlit application  
✅ **Deployment Ready** - Complete with requirements.txt  

### **Files Created**
- 📊 `Air Quality Dashboard.ipynb` - Your learning journey and development
- 🌐 `air_quality_dashboard.py` - Production dashboard application  
- 📋 `requirements.txt` - Dependencies for deployment
- 💾 `global_air_quality_data.csv` - Your collected dataset

### **Skills You've Learned**
🐍 **Python Programming** - APIs, data structures, functions  
📊 **Data Analysis** - Pandas, numpy, data cleaning  
🎨 **Data Visualization** - Plotly, Folium, interactive charts  
🌐 **Web Development** - Streamlit, HTML/CSS basics  
☁️ **Deployment** - Cloud platforms, requirements management  
🔗 **API Integration** - REST APIs, JSON data handling  

### **Next Steps & Enhancements**

#### **Immediate Actions**
1. **Run your dashboard**: `streamlit run air_quality_dashboard.py`
2. **Get your API token**: Visit [aqicn.org/data-platform/token](https://aqicn.org/data-platform/token/)
3. **Deploy online**: Use Streamlit Community Cloud
4. **Add to portfolio**: Include in your resume and LinkedIn

#### **Advanced Features to Add**
- 📧 **Email Alerts** - Notify when air quality gets bad
- 📱 **Mobile App** - Convert to React Native or Flutter
- 🤖 **Machine Learning** - Predict future air quality trends
- 📈 **Historical Data** - Add time series analysis
- 🌦️ **Weather Integration** - Combine with weather data
- 🏥 **Health Impact** - Calculate health costs of pollution

### **Resume-Worthy Project Statement**
*"Developed a real-time Global Air Quality Dashboard processing data from 11,000+ monitoring stations worldwide. Built interactive web application using Python, Streamlit, and REST APIs with geospatial visualizations, automated data pipelines, and cloud deployment. Features include city rankings, pollution trend analysis, and health recommendations based on WHO standards."*

### **Interview Talking Points**
- 🔍 **Problem Solving**: "Identified API changes and adapted to new data sources"
- 📊 **Data Engineering**: "Built ETL pipeline handling real-time data from multiple cities"
- 🎨 **User Experience**: "Designed intuitive dashboard with interactive maps and charts"
- ☁️ **Scalability**: "Deployed on cloud platform with caching for performance"
- 🌍 **Impact**: "Provides accessible air quality information for global health awareness"

---

## 🚀 Ready to Launch Your Career!

You've just built a **professional-grade data science project** that demonstrates:
- Real-world problem solving
- End-to-end development skills  
- Modern technology stack proficiency
- Deployment and scaling knowledge

**This project alone shows you can handle data science roles!** 🎯

Share your dashboard, add it to your portfolio, and start applying for positions. You've got this! 💪